# Install Package

In [1]:
! pip install langchain langchain_experimental langchain-openai langchain-qdrant pypdf sentence-transformers


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_openai import OpenAIEmbeddings

embedding_models = OpenAIEmbeddings(
    openai_api_base = "http://localhost:1234/v1",
    api_key="type-anything-here",
    check_embedding_ctx_length=False
)

In [3]:
query = "This is my first embedding input"
vector = embedding_models.embed_query(query)
print(len(vector), vector)

768 [-0.028102800250053406, 0.0380057729780674, -0.13113786280155182, -0.09799801558256149, 0.008886237628757954, -0.043459586799144745, 0.07824144512414932, 0.01212562806904316, -0.025372019037604332, -0.007348777260631323, -0.05746935307979584, 0.016481773927807808, 0.09535887837409973, 0.06395108252763748, -0.03246573358774185, 0.05639755725860596, 0.04681899771094322, -0.06432924419641495, -0.009352711960673332, 0.016209512948989868, -0.044571805745363235, -0.015060754492878914, -0.03755974397063255, 0.02524692751467228, 0.07082211971282959, 0.029586806893348694, 0.038531940430402756, -0.0025515006855130196, -0.068780317902565, 0.033196624368429184, 0.008204669691622257, 0.005984176881611347, 0.0043681007809937, -0.03510832414031029, -0.022205477580428123, -0.011554487980902195, 0.009527572430670261, 0.02686568722128868, 0.00498193921521306, -0.0027629011310636997, 0.004842245951294899, -0.02176894247531891, 0.0003518109442666173, 0.005985435098409653, -0.0460234172642231, -0.01627

In [4]:
from pypdf import PdfReader
file_path = "./deeplearningbook.pdf"
reader = PdfReader(file_path)
reader.pages[30].extract_text(extraction_mode="layout")

'CHAPTER     1. INTRODUCTION\n\n\nwell-studied  parts  of the  brain  (Olshausen    and  Field,  2005).\n    Neuroscience    has given  us a reason  to hope  that  a single deep  learning  algorithm\ncan  solve many   diﬀerent  tasks.  Neuroscientists   have  found   that ferrets  can learn  to\n“see” with  the  auditory  processing   region  of  their brain  if their brains  are rewired\nto send  visual  signals to  that area  (Von   Melchner   et al., 2000).  This  suggests  that\nmuch   of the  mammalian      brain  might   use a  single algorithm    to solve  most  of the\ndiﬀerent   tasks  that  the  brain  solves.   Before   this hypothesis,    machine    learning\nresearch  was  more   fragmented,    with  diﬀerent  communities     of researchers  studying\nnatural  language   processing,  vision, motion   planning  and  speech  recognition.   Today,\nthese application   communities    are  stillseparate,  but it is common    for deep  learning\nresearch  groups  to study   many

In [5]:
from langchain_experimental.text_splitter import SemanticChunker
text_spliter = SemanticChunker(embedding_models)

docs = text_spliter.create_documents([page.extract_text(extraction_mode="layout") for page in reader.pages])
# docs = text_spliter.create_documents()
print(len(docs))
print(docs[0]) 


Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text

2035
page_content='Deep    Learning



  Ian Goodfellow
  Yoshua  Bengio
  Aaron  Courville'


In [6]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

c:\Users\TrươngĐứcDũng\Desktop\AI\RAG\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [13]:
client = QdrantClient(url="http://localhost:6333")

In [14]:
client.create_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [15]:
print(docs[26].page_content)

. 82
     4.3     Gradient-Based      Optimization       . .


In [16]:
client.upload_points(
    collection_name="my_books",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc.page_content).tolist(), payload=dict(doc)
        )
        for idx, doc in enumerate(docs)
    ],
)

In [19]:
user_query = "I want to know about supervised learning"
hits = client.query_points(
    collection_name="my_books",
    query=encoder.encode(user_query).tolist(),
    limit=3,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'id': None, 'metadata': {}, 'page_content': 'For example,   if we  want  to use  a learning  algorithm\nto perform   object  recognition   from  photographs,     we  need  to specify  which   object\nappears   in each   of the  photos. We   might   do  this with   a numeric    code,  with  0\nsignifying  a person,  1 signifying  a car,  2 signifying  a cat, etc. Often   when   working\nwith  a  dataset   containing   a  design  matrix   of  feature  observations    X,   we  also\nprovide  a  vector  of labels y, with   yi providing  the  label  for example   i. Of  course,  sometimes    the  label may   be  more   than  just  a single  number. For\nexample,    ifwe   want   to train  a  speech   recognition   system   to  transcribe   entire\nsentences,  then  the  label for  each  example   sentence   isa  sequence   of words. Just  as there  is no formal   deﬁnition   of supervised   and   unsupervised    learning,\nthere is no rigid taxonomy    of datasets  or experiences.', 'type'

In [20]:

meta_prompt = f"""
Given the following data, answer the question:
{user_query}
Context:
{hits}
"""

In [22]:
import requests

url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}
payload = {
    "model": "MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF",
    "messages": [
        {"role": "system", "content": "Always answer base on provided context."},
        {"role": "user", "content": meta_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
}

response = requests.post(url, json=payload, headers=headers, stream=True)

if response.status_code == 200:
    for line in response.iter_lines():
        if line:
            decoded_line = line.decode('utf-8')
            print(decoded_line)
else:
    print(f"Request failed with status code: {response.status_code}")


{
  "id": "chatcmpl-kb74xbdo1eeidis4p1zjnk",
  "object": "chat.completion",
  "created": 1731637718,
  "model": "MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF/Mistral-7B-Instruct-v0.3.Q8_0.gguf",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\nSupervised learning is a type of machine learning where a model is trained using labeled data. In this approach, the algorithm learns to make predictions based on input data (x) by adjusting its internal parameters to minimize the difference between predicted output (y_predicted) and actual output (y). The label or output can be a categorical variable such as '0' for person, '1' for car, etc., or a continuous value in regression problems. The labeled data provides information about the correct output for each input, which helps the algorithm to learn patterns and make accurate predictions on unseen data. Examples of supervised learning methods include linear regression, logistic regressio

In [25]:
def chat_with_bot(user_query):
    hits = client.query_points(
        collection_name="my_books",
        query=encoder.encode(user_query).tolist(),
        limit=3,
    ).points
    meta_prompt = f"""
        Given the following data, answer the question:
        {user_query}
        Context:
        {hits}
    """
    url = "http://localhost:1234/v1/chat/completions"
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "model": "MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF",
        "messages": [
            {"role": "system", "content": "Always answer base on provided context."},
            {"role": "user", "content": meta_prompt}
        ],
        "temperature": 0.7,
        "max_tokens": -1,
        "stream": False
    }

    response = requests.post(url, json=payload, headers=headers, stream=True)

    if response.status_code == 200:
        print(response.json()["choices"][0]["message"]["content"])
    else:
        print(f"Request failed with status code: {response.status_code}")


user_question = input("Vui lòng nhập câu hỏi: ")
chat_with_bot(user_question)

Supervised learning is a machine learning technique where the algorithm learns from labeled data. In other words, it uses a dataset containing a design matrix of feature observations X and a vector of labels y to train a model that can make predictions on new, unseen data. The labels provide the correct answers for each example in the dataset, which helps the algorithm to learn the relationship between the features (X) and the label (y).

